Import and setup

In [472]:
import requests
import json
import os
import datetime as dt

import pandas as pd 
import numpy as np
import seaborn as sns

import sqlite3
from pathlib import Path


from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

Path('data\db.db').touch()
conn = sqlite3.connect('data\db.db')
c = conn.cursor()

What are we working with in the games file?

In [473]:
#Only gonna do 2k for messing around
games = pd.read_sql('SELECT * \
                    FROM games \
                    ORDER BY id DESC \
                    LIMIT 15000', con=conn)

#We're going to drop the 'index' column that sqlite creates
games.drop(['index'], axis=1, inplace=True)

#Reorder these so they're actually going up in time
games.sort_values(by='id', ascending=True, inplace=True)
games.reset_index(inplace=True, drop=True)

games.head()

,id,season,date,neutral_site,start_time,tz,full_time,home_team,home_score,away_team,away_score,winner,playoffs,round,series_game,top_seed,bottom_seed,home_wins,away_wins,link
0,2011020587,2011,2012-01-05,0,2012-01-06T00:00:00Z,US/Eastern,REG,PHI,5,CHI,4,PHI,0,N/A,N/A,N/A,N/A,N/A,N/A,nhl.com/gamecenter/chi-vs-phi/2012/01/05/20110...
1,2011020588,2011,2012-01-05,0,2012-01-06T00:30:00Z,US/Eastern,REG,OTT,4,TBL,1,OTT,0,N/A,N/A,N/A,N/A,N/A,N/A,nhl.com/gamecenter/tbl-vs-ott/2012/01/05/20110...
2,2011020589,2011,2012-01-05,0,2012-01-06T01:00:00Z,US/Central,REG,STL,4,EDM,3,STL,0,N/A,N/A,N/A,N/A,N/A,N/A,nhl.com/gamecenter/edm-vs-stl/2012/01/05/20110...
3,2011020590,2011,2012-01-05,0,2012-01-06T01:00:00Z,US/Central,REG,NSH,1,DAL,4,DAL,0,N/A,N/A,N/A,N/A,N/A,N/A,nhl.com/gamecenter/dal-vs-nsh/2012/01/05/20110...
4,2011020591,2011,2012-01-05,0,2012-01-06T03:30:00Z,America/Los_Angeles,OT,LAK,1,PHX,0,LAK,0,N/A,N/A,N/A,N/A,N/A,N/A,nhl.com/gamecenter/phx-vs-lak/2012/01/05/20110...


Adding columns that will help us match our outputs later on

In [474]:
games['home_win'] = np.where(games['home_team'] == games['winner'], 1, 0)
games['home_win_margin'] = games['home_score'] - games['away_score']
games['total_goals'] = games['home_score'] + games['away_score']

games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               15000 non-null  int64 
 1   season           15000 non-null  int64 
 2   date             15000 non-null  object
 3   neutral_site     15000 non-null  int64 
 4   start_time       15000 non-null  object
 5   tz               15000 non-null  object
 6   full_time        15000 non-null  object
 7   home_team        15000 non-null  object
 8   home_score       15000 non-null  int64 
 9   away_team        15000 non-null  object
 10  away_score       15000 non-null  int64 
 11  winner           15000 non-null  object
 12  playoffs         15000 non-null  int64 
 13  round            15000 non-null  object
 14  series_game      15000 non-null  object
 15  top_seed         15000 non-null  object
 16  bottom_seed      15000 non-null  object
 17  home_wins        15000 non-null

Removing columns that we will not end up using in this analysis at this time

In [475]:
drop_cols = ['start_time','tz','link', 'winner']
games.drop(drop_cols, axis=1, inplace=True)

games.head()

,id,season,date,neutral_site,full_time,home_team,home_score,away_team,away_score,playoffs,round,series_game,top_seed,bottom_seed,home_wins,away_wins,home_win,home_win_margin,total_goals
0,2011020587,2011,2012-01-05,0,REG,PHI,5,CHI,4,0,N/A,N/A,N/A,N/A,N/A,N/A,1,1,9
1,2011020588,2011,2012-01-05,0,REG,OTT,4,TBL,1,0,N/A,N/A,N/A,N/A,N/A,N/A,1,3,5
2,2011020589,2011,2012-01-05,0,REG,STL,4,EDM,3,0,N/A,N/A,N/A,N/A,N/A,N/A,1,1,7
3,2011020590,2011,2012-01-05,0,REG,NSH,1,DAL,4,0,N/A,N/A,N/A,N/A,N/A,N/A,0,-3,5
4,2011020591,2011,2012-01-05,0,OT,LAK,1,PHX,0,0,N/A,N/A,N/A,N/A,N/A,N/A,1,1,1


As a sanity check, what % of games are just won by the home team straight up?

In [476]:
home_win_pct = sum(games['home_win'])/games.shape[0]
home_win_pct

0.5416666666666666

We also don't want to take any neutral site games and we don't want any playoff games

In [477]:
games = games.loc[(games['neutral_site']==0) & (games['playoffs']==0)]

drop_cols = ['neutral_site','playoffs','round','series_game','top_seed','bottom_seed','home_wins','away_wins']
games.drop(drop_cols, axis=1, inplace=True)

games.head(10)

,id,season,date,full_time,home_team,home_score,away_team,away_score,home_win,home_win_margin,total_goals
0,2011020587,2011,2012-01-05,REG,PHI,5,CHI,4,1,1,9
1,2011020588,2011,2012-01-05,REG,OTT,4,TBL,1,1,3,5
2,2011020589,2011,2012-01-05,REG,STL,4,EDM,3,1,1,7
3,2011020590,2011,2012-01-05,REG,NSH,1,DAL,4,0,-3,5
4,2011020591,2011,2012-01-05,OT,LAK,1,PHX,0,1,1,1
5,2011020592,2011,2012-01-05,REG,SJS,2,CBJ,1,1,1,3
6,2011020593,2011,2012-01-06,REG,NJD,5,FLA,2,1,3,7
7,2011020594,2011,2012-01-06,REG,PIT,1,NYR,3,0,-2,4
8,2011020595,2011,2012-01-06,REG,CAR,4,BUF,2,1,2,6
9,2011020596,2011,2012-01-06,REG,CHI,0,COL,4,0,-4,4


Now that we have this, we want to pull information on the last couple of games that each team played and use those stats




In [478]:
last_n_games = 10
start_game = 400

#This is for giving teams away wins
switch_dict = {0:1,
               1:0}

for i, game in games.iterrows():

    #We'll not count in the first 100 games of the season to make sure each team has ~5 games in their history already
    #if i < start_game: 
    #    continue

    away = game['away_team']
    home = game['home_team']
    date = game['date']

    #Get the last 
    home_dict = games[((games['home_team']==home) | (games['away_team']==home)) & (games['date'] < date)].iloc[-last_n_games:,]
    away_dict = games[((games['home_team']==away) | (games['away_team']==away)) & (games['date'] < date)].iloc[-last_n_games:,]
    
    home_gpg_last_n = 0
    away_gpg_last_n = 0
    home_point_pct_last_n = 0
    away_point_pct_last_n = 0

    for j, g in home_dict.iterrows():
        if home == g['home_team']:
            home_gpg_last_n += g['home_score']/last_n_games
            home_point_pct_last_n += 2*g['home_win']/last_n_games
            if g['full_time'] != 'REG' and g['home_win'] == 0:
                home_point_pct_last_n += g['home_win']/last_n_games
        else:
            home_gpg_last_n += g['away_score']/last_n_games
            home_point_pct_last_n += 2*switch_dict[g['home_win']]/last_n_games
            if g['full_time'] != 'REG' and g['home_win'] == 1:
                home_point_pct_last_n += switch_dict[g['home_win']]/last_n_games

    for j, g in away_dict.iterrows():
        if away == g['home_team']:
            away_gpg_last_n += g['home_score']/last_n_games
            away_point_pct_last_n += 2*g['home_win']/last_n_games
            if g['full_time'] != 'REG' and g['home_win'] == 0:
                away_point_pct_last_n += g['home_win']/last_n_games
        else:
            away_gpg_last_n += g['away_score']/last_n_games
            away_point_pct_last_n += 2*switch_dict[g['home_win']]/last_n_games
            if g['full_time'] != 'REG' and g['home_win'] == 1:
                away_point_pct_last_n += switch_dict[g['home_win']]/last_n_games

    games.loc[i ,'home_gpg_last_n'] = home_gpg_last_n
    games.loc[i ,'away_gpg_last_n'] = away_gpg_last_n
    games.loc[i ,'home_point_pct_last_n'] = home_point_pct_last_n
    games.loc[i ,'away_point_pct_last_n'] = away_point_pct_last_n

games = games.iloc[start_game-1:,]
games.dropna(axis=0, how='any')

,id,season,date,full_time,home_team,home_score,away_team,away_score,home_win,home_win_margin,total_goals,home_gpg_last_n,away_gpg_last_n,home_point_pct_last_n,away_point_pct_last_n
399,2011020986,2011,2012-03-06,REG,PHI,3,DET,2,1,1,5,3.3,3.1,1.0,1.2
400,2011020987,2011,2012-03-06,OT,WSH,3,CAR,4,0,-1,7,1.9,3.2,0.8,0.8
401,2011020988,2011,2012-03-06,REG,CBJ,3,PHX,2,1,1,5,2.5,2.3,0.8,1.2
402,2011020989,2011,2012-03-06,REG,TBL,3,OTT,7,0,-4,10,2.9,3.6,1.4,1.2
403,2011020990,2011,2012-03-06,REG,STL,5,CHI,1,1,4,6,2.9,2.6,1.4,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2023020474,2023,2023-12-17,REG,CHI,3,VAN,4,0,-1,7,1.8,3.1,0.6,1.2
14996,2023020475,2023,2023-12-17,SO,CAR,1,WSH,2,0,-1,3,3.2,2.5,0.8,0.8
14997,2023020476,2023,2023-12-17,REG,NJD,1,ANA,5,0,-4,6,4.0,2.1,1.6,0.2
14998,2023020477,2023,2023-12-17,REG,COL,6,SJS,2,1,4,8,3.1,3.4,0.8,1.2


In [479]:
games.head()

,id,season,date,full_time,home_team,home_score,away_team,away_score,home_win,home_win_margin,total_goals,home_gpg_last_n,away_gpg_last_n,home_point_pct_last_n,away_point_pct_last_n
399,2011020986,2011,2012-03-06,REG,PHI,3,DET,2,1,1,5,3.3,3.1,1.0,1.2
400,2011020987,2011,2012-03-06,OT,WSH,3,CAR,4,0,-1,7,1.9,3.2,0.8,0.8
401,2011020988,2011,2012-03-06,REG,CBJ,3,PHX,2,1,1,5,2.5,2.3,0.8,1.2
402,2011020989,2011,2012-03-06,REG,TBL,3,OTT,7,0,-4,10,2.9,3.6,1.4,1.2
403,2011020990,2011,2012-03-06,REG,STL,5,CHI,1,1,4,6,2.9,2.6,1.4,1.4


Now we have to standardize the values in the gpg and point percentage columns on normal distributions

In [480]:
gpg_list = games['home_gpg_last_n'].tolist() + games['away_gpg_last_n'].tolist()
ppg_list = games['home_point_pct_last_n'].tolist() + games['away_point_pct_last_n'].tolist()

gpg_mean = np.average(gpg_list)
gpg_sd = np.std(gpg_list)
ppg_mean = np.average(ppg_list)
ppg_sd = np.std(ppg_list)

games['home_gpg_last_n'] = (games['home_gpg_last_n'] - gpg_mean)/gpg_sd
games['away_gpg_last_n'] = (games['away_gpg_last_n'] - gpg_mean)/gpg_sd
games['home_point_pct_last_n'] = (games['home_point_pct_last_n'] - ppg_mean)/ppg_sd
games['away_point_pct_last_n'] = (games['away_point_pct_last_n'] - ppg_mean)/ppg_sd

#games['home_gpg_diff'] = games['home_gpg_last_n'] - games['away_gpg_last_n']
#games['home_ppg_diff'] = games['home_point_pct_last_n'] - games['away_point_pct_last_n']

#We'll also normalize the columns for win margin and total_goals
games[['home_win_margin']] = StandardScaler().fit_transform(games[['home_win_margin']])
games[['total_goals']] = StandardScaler().fit_transform(games[['total_goals']])

#Drop the helper columns we made along hte way
#helper_cols = ['home_gpg_last_n','away_gpg_last_n','home_point_pct_last_n','away_point_pct_last_n']
#games.drop(helper_cols,axis=1, inplace=True)

#Drop the other columns that are just giving us info at this point and nothing predictive
info_cols = ['id','season','date','home_team','full_time','home_score','away_score','away_team']
games.drop(info_cols,axis=1,inplace=True)

In [481]:
games.head()

,home_win,home_win_margin,total_goals,home_gpg_last_n,away_gpg_last_n,home_point_pct_last_n,away_point_pct_last_n
399,1,0.299399,-0.366850,0.626882,0.305819,0.004654,0.571566
400,0,-0.511959,0.511307,-1.620555,0.466350,-0.562257,-0.562257
401,1,0.299399,-0.366850,-0.657368,-0.978430,-0.562257,0.571566
402,0,-1.728995,1.828541,-0.015243,1.108475,1.138478,0.571566
403,1,1.516435,0.072228,-0.015243,-0.496837,1.138478,1.138478


What do the correlations look like?

In [482]:
games.corr()['home_win']

home_win                 1.000000
home_win_margin          0.857358
total_goals              0.010031
home_gpg_last_n          0.059225
away_gpg_last_n         -0.077791
home_point_pct_last_n    0.078513
away_point_pct_last_n   -0.080192
Name: home_win, dtype: float64

Stronger correlation with the ppg differential than with the gpg differential

In [483]:
x = games[['home_gpg_last_n','away_gpg_last_n','home_point_pct_last_n','away_point_pct_last_n']]
y = games['home_win']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle=True)

In [484]:
linear_model = LinearRegression().fit(x_train, y_train)
y_pred = linear_model.predict(x_test)
y_pred = [round(y) for y in y_pred]

In [485]:
accuracy_score(y_test, y_pred)

0.562035661218425

How much better is this than the just home team each time?

In [486]:
(accuracy_score(y_test, y_pred)/home_win_pct - 1)*100

3.7604297634015404

In [487]:
f1_score(y_test, y_pred)

0.6613042229244469